Step 1: 

In [14]:
Downloaded MariaDB 10.5

CREATE DATABASE cov;

USE cov;

CREATE TABLE Assaysb (
SMILES VARCHAR(255),
CID VARCHAR(255) PRIMARY KEY,
canonical_CID VARCHAR(255),
r_inhibition_at_20_uM FLOAT,
r_inhibition_at_50_uM FLOAT,
r_avg_IC50 FLOAT,
f_inhibition_at_20_uM FLOAT,
f_inhibition_at_50_uM FLOAT,
f_avg_IC50 FLOAT,
f_avg_pIC50 FLOAT,
relative_solubility_at_20_uM FLOAT,
relative_solubility_at_100_uM FLOAT,
trypsin_IC50 FLOAT,
NMR_std_ratio FLOAT,
acrylamide VARCHAR(255),
chloroacetamide VARCHAR(255),
series VARCHAR(255),
frag_id VARCHAR(255)
);

LOAD DATA LOCAL INFILE 'C:/Users/HP ZBOOK/Desktop/activity_data.csv' INTO TABLE Assaysb 
FIELDS TERMINATED BY ',' ENCLOSED BY '' 
LINES TERMINATED BY '\n' 
IGNORE 1 ROWS;

ALTER TABLE ASSAYS
DROP COLUMN SMILES;

CREATE INDEX CIDIDX ON assays(CID);

CREATE TABLE compounds (
    SMILES TEXT NOT NULL,
    CID VARCHAR(255) PRIMARY KEY,
    canonical_CID VARCHAR(255)
);


LOAD DATA LOCAL INFILE 'C:/Users/HP ZBOOK/Desktop/activity_data.csv' INTO TABLE COMPOUNDS
FIELDS TERMINATED BY ',' ENCLOSED BY '' 
LINES TERMINATED BY '\n' 
IGNORE 1 ROWS
(SMILES,CID,canonical_CID);


ALTER TABLE assays ADD CONSTRAINT CID
FOREIGN KEY(CID) REFERENCES compounds (CID);


test: 
SELECT compounds.SMILES, assays.f_avg_IC50, assays.f_avg_pIC50
FROM compounds
JOIN assays ON compounds.CID = assays.CID
WHERE compounds.CID = 'LUO-POS-e1dab717-11';






SyntaxError: invalid syntax (1423913488.py, line 1)

Step 2:

In [15]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Descriptors

# Read the compounds data from the CSV file
compounds_df = pd.read_csv('C:/Users/HP ZBOOK/Desktop/activity_data.csv')

In [16]:
# Create new columns for the molecular descriptors
compounds_df['H_bond_donors'] = None
compounds_df['H_bond_acceptors'] = None
compounds_df['MW'] = None
compounds_df['logP'] = None

# Iterate over the compounds and calculate the molecular descriptors
for index, row in compounds_df.iterrows():
    # Get the SMILES string of the compound
    smiles = row['SMILES']
    # Create a rdkit molecule object from the SMILES string
    mol = Chem.MolFromSmiles(smiles)
    # Calculate the number of hydrogen bond donors
    h_bond_donors = Chem.Lipinski.NumHDonors(mol)
    # Calculate the number of hydrogen bond acceptors
    h_bond_acceptors = Chem.Lipinski.NumHAcceptors(mol)
    # Calculate the molecular weight
    mw = Descriptors.MolWt(mol)
    # Calculate the octanol-water partition coefficient
    logp = Descriptors.MolLogP(mol)
    # Add the molecular descriptors to the corresponding columns in the dataframe
    compounds_df.at[index, 'H_bond_donors'] = h_bond_donors
    compounds_df.at[index, 'H_bond_acceptors'] = h_bond_acceptors
    compounds_df.at[index, 'MW'] = mw
    compounds_df.at[index, 'logP'] = logp

In [22]:
import mariadb

mydb = mariadb.connect(
        user="root",
        password="zhaosimiao1120",
        host="127.0.0.1",
        port=3306,
        database="cov"

    )

In [23]:
cursor = mydb.cursor()

In [29]:
print(len(compounds_df))

2062


In [30]:
cursor.execute("ALTER TABLE compounds ADD COLUMN HBD INTEGER")
cursor.execute("ALTER TABLE compounds ADD COLUMN HBA INTEGER")
cursor.execute("ALTER TABLE compounds ADD COLUMN MW FLOAT")
cursor.execute("ALTER TABLE compounds ADD COLUMN logP FLOAT")

# Use a for loop to insert the molecular descriptors into the new columns
for i in range(len(compounds_df)):
    cursor.execute("UPDATE compounds SET HBD = %s, HBA = %s, MW = %s, logP = %s WHERE CID = %s", (compounds_df['H_bond_donors'][0], compounds_df['H_bond_acceptors'][i], compounds_df['MW'][i], compounds_df['logP'][i], compounds_df['CID'][i]))

# Commit the changes to the database
mydb.commit()

In [32]:
# Write and execute the query to extract the desired data
query = "SELECT compounds.SMILES, compounds.HBD, compounds.HBA, compounds.MW, compounds.logP, assays.f_avg_IC50 FROM compounds JOIN assays ON compounds.CID = assays.CID WHERE compounds.HBD <= 5 AND compounds.HBA <= 10 AND compounds.MW <= 500 AND compounds.logP <= 5"
cursor.execute(query)

# Fetch and print the results
results = cursor.fetchall()
print(len(results))
for result in results:
    print(result)

1539
('Cn1ccc(C(=O)NC[C@@H]2CCCO2)n1', 1, 4, 209.249, 0.3289, 0.0)
('CS(=O)(=O)c1ccc(N2CCNCC2)cc1', 1, 4, 240.328, 0.4997, 99.5)
('CN(C(=O)Cn1nnc2ccccc21)c1ccc(Oc2ccc(C(F)(F)F)cn2)cc1', 1, 6, 427.386, 4.3004, 37.8324)
('Cc1ccc(S(=O)(=O)N2CCN(C(=O)CCl)CC2)cc1', 1, 3, 316.81, 1.06672, 21.0262)
('N#Cc1nccn1CCc1cccc(Cl)c1', 1, 3, 231.686, 2.65088, 99.5)
('N#Cc1nccc(Cl)n1', 1, 3, 139.545, 1.00168, 99.5)
('COc1cc(OC)[n+](C)c(C#N)n1', 1, 4, 180.187, -0.20502, 99.5)
('C[n+]1ccn(CCc2cccc(Cl)c2)c1C#N', 1, 2, 246.721, 2.08038, 99.5)
('CC1(C)OB(c2cnc(C#N)nc2)OC1(C)C', 1, 5, 231.064, 0.64748, 99.5)
('COc1cc(OC)nc(C#N)n1', 1, 5, 165.152, 0.36548, 99.5)
('CC(Cn1ccnc1C#N)c1cccc(Cl)c1', 1, 3, 245.713, 3.21188, 99.5)
('N#Cc1nccn1CCCc1cccc(Cl)c1', 1, 3, 245.713, 3.04098, 99.5)
('Cc1cc(C)nc(C#N)n1', 1, 3, 133.154, 0.96512, 99.5)
('N#Cc1ncc(Cl)cn1', 1, 3, 139.545, 1.00168, 99.5)
('N#Cc1ncc(Br)cn1', 1, 3, 183.996, 1.11078, 99.5)
('Cc1cc(C)[n+](C)c(C#N)n1', 1, 2, 148.189, 0.39462, 99.5)
('C[n+]1ccn(CCCc2cccc

There are still some low affinity values

In [51]:
query = "SELECT compounds.SMILES, compounds.HBD, compounds.HBA, compounds.MW, compounds.logP, assays.f_avg_IC50 FROM compounds JOIN assays ON compounds.CID = assays.CID GROUP BY assays.CID HAVING SUM(CASE WHEN compounds.HBD > 5 THEN 1 ELSE 0 END + CASE WHEN compounds.HBA > 10 THEN 1 ELSE 0 END + CASE WHEN compounds.MW > 500 THEN 1 ELSE 0 END + CASE WHEN compounds.logP > 5 THEN 1 ELSE 0 END) <= 1;"
cursor.execute(query)
# Fetch and print the results
results2 = cursor.fetchall()
print(len(results2))
for result in results2:
    print(result)

1981
('Cn1ccc(C(=O)NC[C@@H]2CCCO2)n1', 1, 4, 209.249, 0.3289, 0.0)
('CS(=O)(=O)c1ccc(N2CCNCC2)cc1', 1, 4, 240.328, 0.4997, 99.5)
('CN(C(=O)Cn1nnc2ccccc21)c1ccc(Oc2ccc(C(F)(F)F)cn2)cc1', 1, 6, 427.386, 4.3004, 37.8324)
('Cc1ccc(S(=O)(=O)N2CCN(C(=O)CCl)CC2)cc1', 1, 3, 316.81, 1.06672, 21.0262)
('N#Cc1nccn1CCc1cccc(Cl)c1', 1, 3, 231.686, 2.65088, 99.5)
('N#Cc1nccc(Cl)n1', 1, 3, 139.545, 1.00168, 99.5)
('COc1cc(OC)[n+](C)c(C#N)n1', 1, 4, 180.187, -0.20502, 99.5)
('C[n+]1ccn(CCc2cccc(Cl)c2)c1C#N', 1, 2, 246.721, 2.08038, 99.5)
('CC1(C)OB(c2cnc(C#N)nc2)OC1(C)C', 1, 5, 231.064, 0.64748, 99.5)
('COc1cc(OC)nc(C#N)n1', 1, 5, 165.152, 0.36548, 99.5)
('CC(Cn1ccnc1C#N)c1cccc(Cl)c1', 1, 3, 245.713, 3.21188, 99.5)
('N#Cc1nccn1CCCc1cccc(Cl)c1', 1, 3, 245.713, 3.04098, 99.5)
('Cc1cc(C)nc(C#N)n1', 1, 3, 133.154, 0.96512, 99.5)
('N#Cc1ncc(Cl)cn1', 1, 3, 139.545, 1.00168, 99.5)
('N#Cc1ncc(Br)cn1', 1, 3, 183.996, 1.11078, 99.5)
('Cc1cc(C)[n+](C)c(C#N)n1', 1, 2, 148.189, 0.39462, 99.5)
('C[n+]1ccn(CCCc2cccc

Yes. The compounds violate no more than one rule are more but not a lot. 